In [8]:
# Required by Google Colab
!pip install qiskit qiskit_nature pyscf --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 33.6 MB/s eta 0:00:00


### Define electronic molecule of $BeH_{2}$

In [5]:
import time
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer

# Define molecule BeH2
distance = 1.7
molecule = Molecule(geometry=[['H', [0., 0., -distance]],
                              ['Be', [0., 0., 0.]],
                              ['H', [0., 0., distance]],
                              ], charge=0, multiplicity=1)

# set up a PySCF driver at bondlength 1.7 Angstrom (distance)
driver = ElectronicStructureMoleculeDriver(molecule, basis='sto3g', driver_type=ElectronicStructureDriverType.PYSCF)

freezeCoreTransformer = FreezeCoreTransformer(True)
problem = ElectronicStructureProblem(driver, transformers=[freezeCoreTransformer])

# JordanWignerMapper()
mapper = ParityMapper()

qubit_converter = QubitConverter(mapper, two_qubit_reduction=True, z2symmetry_reduction='auto')

### VQE

In [11]:
%%time
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import SLSQP
from qiskit_nature.algorithms import VQEUCCFactory

params_history = []
def callback(xk):
    return params_history.append(xk)

optimizer = SLSQP(maxiter=50, callback=callback)
backend_name = 'aer_simulator_statevector' # Wall time: maxiter=1 -> 7.77 s, maxiter=50 -> 16.9 s
quantum_instance = QuantumInstance(backend = Aer.get_backend(backend_name))
solver = VQEUCCFactory(quantum_instance,optimizer=optimizer)
gs_solver = GroundStateEigensolver(qubit_converter, solver)
result = gs_solver.solve(problem)
print(result)

<timed exec>:14: DeprecationWarning: VQEUCCFactory Constructor: quantum_instance is no longer a positional argument as of version 0.4 and will be removed no sooner than 3 months after the release. Instead use it as a keyword argument


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -18.173645693686
  - computed part:      -3.488773764541
  - FreezeCoreTransformer extracted energy part: -14.684871929146
~ Nuclear repulsion energy (Hartree): 2.6458860546
> Total ground state energy (Hartree): -15.527759639086
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [None  None  None]
    - computed part:      [None  None  None]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [None  None  None]  Total: None
                 (debye): [None  None  None]  Total: None
 
CPU times: user 15 s, sys: 4.11 s, total: 19.1 s
Wall time: 16.9 s
